In [4]:

#Hi anjali this side
import numpy as np
from pyspark.sql import SQLContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
# import pandas.Series.tolist
import pandasql as ps
import boto3
import pyodbc
import io
from datetime import date,timedelta
from IPython.display import display
import json
import random
import time
import re
### Custom Lib
pd.options.mode.chained_assignment = None
from pyspark.sql.types import IntegerType,StructType,StructField,StringType
from pyspark.sql.functions import lit,array, create_map, struct
from pyspark.sql.functions import col, concat, regexp_extract, when,max as max_,min as min_
from pyspark.sql.functions import date_format
from pyspark.sql.functions import udf,col
from datetime import datetime,timedelta
from pymongo import MongoClient
import traceback
import ftplib



An error was encountered:
You need to have at least 1 client created to execute commands.


In [5]:
def errorcategorycode(errtext):
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=10.100.11.5;DATABASE=datalake;UID=datalake_prod;PWD=uqkHej49KENG')
    querystring_details = "select * from etl.error_category with(nolock)"
    error_code='MISC'
    try:
        dataframe_details = pd.read_sql(querystring_details, cnxn)
        if len(dataframe_details) > 0:
            for row in range(0,len(dataframe_details)):
                error_keywords = dataframe_details['error_category_text'][row].split("|")
                if any(x in errtext for x in error_keywords):
                    error_code = dataframe_details['error_category_code'][row]
                    break
    except:
        pass
    cnxn.close()
    return error_code

An error was encountered:
You need to have at least 1 client created to execute commands.


In [ ]:
sc.addPyFile("s3n://c1-data-lake/ETL/ETL_Dependency/etlconfig.py")
sc.addPyFile("s3://c1-data-lake/ETL/UDF/prod/Udf.py")
sc.addPyFile("s3n://c1-data-lake/ETL/psqlconfig/psqlconfig.py")

import Udf
import etlconfig
import psqlconfig

conf = SparkConf()

conf.set("spark.serializer", "org.apache.spark.serializer.KyroSerializer")
conf.set("spark.executor.extraJavaOptions", "-XX:+UseParallelGC -XX:+UseParallelOldGC")
conf.set("spark.jars", "s3://c1-data-lake/jars/postgresql-42.3.1.jar")
conf.set('spark.executor.extraClassPath', "s3://c1-data-lake/jars/postgresql-42.3.1.jar")
conf.set('spark.driver.extraClassPath', 's3://c1-data-lake/jars/postgresql-42.3.1.jar')


spark = SparkSession.builder\
                    .appName("Sparkonlineclient") \
                    .config(conf=conf) \
                    .getOrCreate()


sc=spark.sparkContext
sqlCtx = SQLContext(sc)
sqlCtx.clearCache()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
sparkread = spark.read
todaydate = date.today()
date_formatsmall=todaydate.strftime("%y%m%d")
# todaydate = date.today()
todaydate = date.today() - timedelta(days=1)
print(todaydate)
count=0
bucket='c1-data-lake'
s3_resource = boto3.client('s3')
